In [8]:
# 1. Dane z "wszystko"
# 2. Zamiana obrazów na Base64
# 3. Wysyłanie requestów (1 na 10 sekund)
# 4. Zapisywanie wyników
import requests
import base64
import time
import os

In [9]:
# Wczytanie danych
auth_path = "../Dane/wszystko/authentic"
spoof_path = "../Dane/wszystko/spoof"

auth_files = os.listdir(auth_path)
spoof_files = os.listdir(spoof_path)

auth_files = [os.path.join(auth_path, file) for file in auth_files]
spoof_files = [os.path.join(spoof_path, file) for file in spoof_files]

print("Ilość zdjęć autentycznych: ", len(auth_files))
print("Ilość zdjęć podstawionych: ", len(spoof_files))

Ilość zdjęć autentycznych:  252
Ilość zdjęć podstawionych:  235


In [10]:
# POST https://face-liveness-detection3.p.rapidapi.com/api/liveness_base64 HTTP/1.1
# Accept: application/json
# Content-Type: application/json
# x-rapidapi-ua: RapidAPI-Playground
# x-rapidapi-key: 5719b3ffa5msh94945ab740d3217p197d75jsn2f970af3cecc
# x-rapidapi-host: face-liveness-detection3.p.rapidapi.com
# specificMethodHeaders: [object Object]
# {"image":"

# Przygotowanie requestów
url = "https://face-liveness-detection3.p.rapidapi.com/api/liveness_base64"
key = "5719b3ffa5msh94945ab740d3217p197d75jsn2f970af3cecc"
content_type = "application/json"
accept = "application/json"
ua = "RapidAPI-Playground"
host = "face-liveness-detection3.p.rapidapi.com"

i = 0

for img in auth_files:
    with open(img, "rb") as img_file:
        img_base64 = base64.b64encode(img_file.read()).decode("utf-8")
        
        headers = {
            "x-rapidapi-key": key,
            "x-rapidapi-host": host,
            "x-rapidapi-ua": ua,
            "Content-Type": content_type,
            "Accept": accept
        }
        
        data = {
            "image": img_base64
        }
        
        response = requests.post(url, headers=headers, json=data)
        
        # Zapis do pliku
        with open(f"wyniki/auth{i}.txt", "w") as results:
            results.write(f"{img}: {response.text}\n")
            
        i += 1
        
        time.sleep(10)

In [11]:
i = 0

for img in spoof_files:
    with open(img, "rb") as img_file:
        img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

        headers = {
            "x-rapidapi-key": key,
            "x-rapidapi-host": host,
            "x-rapidapi-ua": ua,
            "Content-Type": content_type,
            "Accept": accept
        }

        data = {
            "image": img_base64
        }

        response = requests.post(url, headers=headers, json=data)

        # Zapis do pliku
        with open(f"wyniki/spoof{i}.txt", "w") as results:
            results.write(f"{img}: {response.text}\n")

        i += 1

        time.sleep(10)

In [12]:
# Analiza wyników
all_results = os.listdir("wyniki")
auth_acc = 0
auth_total = 0
spoof_acc = 0
spoof_total = 0
no_face_detected_total = 0

for result in all_results:
    with open(f"wyniki/{result}", "r") as file:
        content = file.read()
        
        if "no face detected!" in content:
            no_face_detected_total += 1
            continue
        
        if "auth" in result:
            if "genuine" in content:
                auth_acc += 1
            auth_total += 1
        else:
            if "small face" in content or "spoof" in content:
                spoof_acc += 1
            spoof_total += 1
            
print("Dla zdjęć autentycznych:")
print(f"Poprawnie sklasyfikowano {auth_acc} na {auth_total}")
print("Dla zdjęć podstawionych:")
print(f"Poprawnie sklasyfikowano {spoof_acc} na {spoof_total}")
print(f"Nie wykryto twarzy na {no_face_detected_total} zdjęciach")

Dla zdjęć autentycznych:
Poprawnie sklasyfikowano 8 na 247
Dla zdjęć podstawionych:
Poprawnie sklasyfikowano 145 na 145
Nie wykryto twarzy na 95 zdjęciach
